In [1]:
import pandas as pd
df = pd.read_csv("glassdoor_jobs.csv")

In [2]:
df.head()

,Unnamed: 0,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\r\n\r\nAt Hopper, we’re on a miss...",3.5,Hopper\r\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\r\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
2,2,2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\r\n\r\nhttps://www.decode-m.com/\r\n\...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True
3,3,3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\r\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1
4,4,4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\r\nDescripti...",3.4,United Entertainment Group\r\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1


In [3]:
df.shape

(3909, 17)

### Salary Parsing
We want: 
- the company name text only
- state field
- Age of Company
- Parsing of job description(python, etc.)

In [4]:
df['hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)

In [5]:
df['employer_provied'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)

In [6]:
df = df[df['Salary Estimate'] != '-1']

In [7]:
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])

In [8]:
salary

0       $111K-$181K 
1       $111K-$181K 
2       $111K-$181K 
3       $111K-$181K 
4       $111K-$181K 
            ...     
3904     $55K-$112K 
3905     $55K-$112K 
3906     $55K-$112K 
3907     $55K-$112K 
3908     $55K-$112K 
Name: Salary Estimate, Length: 3909, dtype: object

In [9]:
df = df[df['Competitors'] != '-1']

In [10]:
df.shape

(1149, 19)

In [11]:
minus_Kd = salary.apply(lambda x: x.replace('K','').replace('$',''))

In [12]:
minus_Kd

0       111-181 
1       111-181 
2       111-181 
3       111-181 
4       111-181 
          ...   
3904     55-112 
3905     55-112 
3906     55-112 
3907     55-112 
3908     55-112 
Name: Salary Estimate, Length: 3909, dtype: object

In [13]:
min_hr = minus_Kd.apply(lambda x: x.lower().replace('per hour', '').replace('employer provided salary:', ''))

In [14]:
df['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))

In [15]:
df['min_salary'].dtype

dtype('int64')

In [16]:
df['max_salary'] = min_hr.apply(lambda x: int(x.split('-')[1]))

In [17]:
df['avg_salary'] = (df.min_salary + df.max_salary)/2

### Company name text only

In [19]:
 df['company_txt'] = df.apply(lambda x: x['Company Name'] if x['Rating'] < 0 else x['Company Name'][:-3], axis=1)

## State Field(The State the job is in)

In [22]:
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1])

In [26]:
df.job_state.value_counts()

 TX                361
 CA                342
 IL                119
 PA                100
 NY                 90
 AZ                 76
 OH                 43
 NJ                  7
 FL                  5
 United Kingdom      4
 DE                  2
Name: job_state, dtype: int64

In [29]:
df['same_state'] = df.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis = 1)

## Age of Company

In [30]:
df['age'] = df.Founded.apply(lambda x: x if x < 1 else 2021 - x)

## Parsing of job descriptions

In [33]:
df['Job Description']

3       Sapphire Digital seeks a dynamic and driven mi...
4       Director, Data Science - (200537)\r\nDescripti...
5       Job Brief\r\n\r\nThe ideal candidate will have...
11      PulsePoint™, a global programmatic advertising...
12      Medidata: Conquering Diseases Together\r\n\r\n...
                              ...                        
3895    Help us transform patients' lives.\r\n\r\nAt U...
3899    Sr Data Analyst\r\n\r\n\r\nJob Description\r\n...
3900    ICS is working with a Fortune 500, industry le...
3903    ApTask is a leading staffing and recruitment c...
3908    Help us transform patients' lives.\r\nAt UCB, ...
Name: Job Description, Length: 1149, dtype: object

### Python

In [34]:
df['python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

In [35]:
df.python.value_counts()

1    706
0    443
Name: python, dtype: int64

### R Studio

In [38]:
df['r_studio'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)

In [39]:
df.r_studio.value_counts()

0    1148
1       1
Name: r_studio, dtype: int64

## Spark

In [40]:
df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)

In [41]:
df.spark.value_counts()

0    827
1    322
Name: spark, dtype: int64

## AWS

In [42]:
df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)

In [43]:
df.aws.value_counts()

0    811
1    338
Name: aws, dtype: int64

## Excel

In [44]:
df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

In [45]:
df.excel.value_counts()

1    603
0    546
Name: excel, dtype: int64

In [48]:
df_out = df.drop(['Unnamed: 0'], axis=1)

In [49]:
df_out.to_csv('salary_data_cleaned.csv', index = False)